In [1]:
# mnist_cnn_deep
# MNIST and Convolutional Neural Network
# L1,L2 : conv2d + relu + max_pool 
# L3 : FC(Fully Connected Layer)

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
tf.random.set_seed(5)

In [5]:
# mnist 데이터 가져오기
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [6]:
# one-hot 인코딩
nb_classes = 10 # 분류 class의 개수

Y_one_hot = tf.one_hot(y_train, nb_classes) # (60000,)
Y_one_hot.shape                             # (60000,10)

TensorShape([60000, 10])

In [8]:
x_train[0].dtype

dtype('uint8')

In [20]:
# x를 실수형으로 변환
x_train = tf.cast(x_train,dtype=tf.float32)
x_test = tf.cast(x_test,dtype=tf.float32)
print(x_train.shape)

(60000, 28, 28)


In [21]:
# x의 shape을 4차원으로 변환
x_img = tf.reshape(x_train,[-1,28,28,1])
print(x_img.shape)

(60000, 28, 28, 1)


In [53]:
# Layer 1 : conv2d - relu - max_pool
# (?, 28, 28, 1) --> (?, 14, 14, 32)

# <1> conv2d
# L1 input image shape : (?, 28, 28, 1)
# filter : (3,3,1,32), 필터 32개
# strides : (1,1,1,1), padding='SAME'
# 출력 이미지 : (28+2 - 3)/1 + 1 = 28
# (?, 28, 28, 1) --> (?, 28, 28, 32)

W1 = tf.Variable(tf.random.normal([3,3,1,32]),name="weight1")

def L1_conv2d(X):
    return tf.nn.conv2d(X,W1,strides=[1, 1, 1, 1], padding="SAME")

# <2> relu
def L1_relu(X):
    return tf.nn.relu(L1_conv2d(X)) # shape 변화 X

# <3> max_pool
# input image : (?, 28, 28, 32)
# ksize : (1,2,2,1), strides : (1,2,2,1), padding='SAME'
# 출력 이미지 : (28+1 - 2)/2 + 1 = 14
#  (?, 28, 28, 32) -->  (?, 14, 14, 32)
def L1_Maxpool(X):
    return tf.nn.max_pool(L1_relu(X),ksize=[1,2,2,1],strides=[1,2,2,1], padding="SAME")

In [54]:
# Layer 2 : conv2d - relu - max_pool - flatten
# (?, 14, 14, 32) --> (?, 7*7*64)

# <1> conv2d
# L2 input image shape : (?, 14, 14, 32)
# filter : (3,3,32,64), 필터 64개
# strides : (1,1,1,1), padding='SAME'
# 출력 이미지 : (14+2 - 3)/1 + 1 = 14
# (?, 14, 14, 32) --> (?, 14,14,64)

W2 = tf.Variable(tf.random.normal([3,3,32,64]),name="weight2")

def L2_conv2d(X):
    return tf.nn.conv2d(L1_Maxpool(X),W2,strides=[1, 1, 1, 1], padding="SAME")

# <2> relu
def L2_relu(X):
    return tf.nn.relu(L2_conv2d(X)) # shape 변화 X

# <3> max_pool
# input image : (?, 14, 14, 64)
# ksize : (1,2,2,1), strides : (1,2,2,1), padding='SAME'
# 출력 이미지 : (14+1 - 2)/2 + 1 = 7
#  (?, 14, 14, 64) -->  (?, 7, 7, 64)
def L2_Maxpool(X):
    return tf.nn.max_pool(L2_relu(X),ksize=[1,2,2,1],strides=[1,2,2,1], padding="SAME")

# <4> flatten layer : 다차원 배열을 2차원으로 변환하여 FC layer에 전달한다
def L2_flat(X):
    return tf.reshape(L2_Maxpool(X), [-1,7*7*64])

In [55]:
# Layer 3 : FC(Fully Connected Layer)
# (?,7*7*64) * (  7*7*64,  10 ) = (?,10)
W3 = tf.Variable(tf.random.normal([7*7*64, nb_classes]),name='weight3')
b = tf.Variable(tf.random.normal([nb_classes]),name='bias')

In [60]:
# 예측 함수(hypothesis) : H(x) = softmax(W*X + b)
def logits(X):
    return tf.matmul(L2_flat(X),W3) + b
    
def hypothesis(X):
    return tf.nn.softmax(logits(X))

In [61]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [62]:
x_train.shape

TensorShape([60000, 28, 28])

In [64]:
# batch 사이즈로 나누어 학습, 효율적이며 학습 시간 단축
training_epoch = 50
batch_size = 1200

print("-----------start!!--------------")
for epoch in range (training_epoch): # 50회

    avg_cost = 0
    
    # 50 = 60000/1200
    total_batch = int(x_train.shape[0]/batch_size)
    for k in range(total_batch): # 50 회
        batch_xs = x_train[k*batch_size:batch_size*(k+1)] # 1200개 x데이터
        batch_ys = Y_one_hot[k*batch_size:batch_size*(k+1)] # 1200개 y데이터

        # X값의 shape을 4차원으로 변환
        X_img = tf.reshape(batch_xs,[-1,28,28,1])
        def cost_func_batch():
            cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits(X_img),
                                                             labels = batch_ys)
            cost = tf.reduce_mean(cost_i)
            return cost
            
        optimizer.minimize(cost_func_batch,var_list=[W1,W2,W3,b])
        avg_cost += cost_func_batch().numpy()/total_batch
    print('Epoch:','%04d'%(epoch + 1),'cost:','{:.9f}'.format(avg_cost))
print("-----------Fin!!--------------")

    

-----------start!!--------------
Epoch: 0001 cost: 21572.957773438
Epoch: 0002 cost: 3155.678679199
Epoch: 0003 cost: 1888.005522461
Epoch: 0004 cost: 1325.722739258
Epoch: 0005 cost: 996.843026733
Epoch: 0006 cost: 780.098104248
Epoch: 0007 cost: 621.970330200
Epoch: 0008 cost: 524.722174072
Epoch: 0009 cost: 457.456445618
Epoch: 0010 cost: 389.688021545
Epoch: 0011 cost: 337.727833557
Epoch: 0012 cost: 284.673877563
Epoch: 0013 cost: 245.595002899
Epoch: 0014 cost: 208.207411346
Epoch: 0015 cost: 185.912599182
Epoch: 0016 cost: 169.433121872
Epoch: 0017 cost: 142.446161804
Epoch: 0018 cost: 125.435796127
Epoch: 0019 cost: 109.810541954
Epoch: 0020 cost: 100.607365189
Epoch: 0021 cost: 88.225869141
Epoch: 0022 cost: 76.783376751
Epoch: 0023 cost: 71.982066345
Epoch: 0024 cost: 59.648774123
Epoch: 0025 cost: 57.705766869
Epoch: 0026 cost: 54.313736405
Epoch: 0027 cost: 47.871068583
Epoch: 0028 cost: 44.148553438
Epoch: 0029 cost: 40.213280935
Epoch: 0030 cost: 33.271895990
Epoch: 0031 

In [65]:
# 정확도 측정 : accuracy computation

# y_test 값의 one-hot 인코딩
Y_one_hot = tf.one_hot(y_test,nb_classes)    # (10000,10)
# print(Y_one_hot.shape)                       # (10000,10)  , (2차원)

# tf.argmax() : 값이 가장 큰 요소의 인덱스 값을 반환
def predict(X):
    return tf.argmax(hypothesis(X),axis=1)

# X값의 shape을 4차원으로 변환
X_img = tf.reshape(x_test,[-1,28,28,1])
        
correct_predict = tf.equal(predict(X_img),tf.argmax(Y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_predict, dtype = tf.float32))
print("Accuracy:",accuracy.numpy()) # Accuracy: 0.9534

#예측
print('***** Predict')
pred = predict(X_img).numpy()
print(pred,y_test)

Accuracy: 0.9805
***** Predict
[7 2 1 ... 4 5 6] [7 2 1 ... 4 5 6]
